# Inverted Index for Boolean Search

Simple Inverted Index for news retrieval (having to choose between linked lists or varying arrays for posting)


#### Modeling and design decisions

- [x] Make cells independent one another
- [x] Use ids as indexes and in filenames as ident
- [x] Make title important for retrieval
- [x] Use suitable filename for unix systems
- [x] Make corpora compatible to the nltk library



> Although provided by the NLTK library, this document do not apply "cleansing" of the portuguese. For example, in exclusion of stoping words, synonyms words, and other usage.

#### The Dataset (extracted from a paid brazilian portal)

In [1]:
import pandas as pd
politicnews = pd.read_csv('./dataset/news__ptbr.csv', 
                   names=['title', 'content','id'], 
                   header=0)

##### Visualization of the dataset
```python
from googletrans import Translator as t # limit: 15k

h = politicnews.head(5); vis = pd.DataFrame(); CHAR=50

#Takes up to ~5 seconds for Google Translator response
en=lambda x: t().translate(x, 
                           src="pt", 
                           dest="en").text[:CHAR]

vis["title"] = h["title"].map(en)
vis["content"] = h["content"].map(en)
vis['id'] = h['id']

print(vis) 
```


| title                         | content                            | id   |
|-----------------------------  |------------------------------------|------|
| 2018 Looking Like 2006        | ...                                  | 1004 |
| Social Division May benefit ..   | Brazil defends political scientist | 1032 |
| ...                         | ...                              | ...  |

#### Details and requirements
By having unlimited and measurable alocated size, linked lists was chosen. Raymond Hettinger’s [OrderedSet](https://orderedset.readthedocs.io/en/latest/) is suitable for fast deletion. It is doubly linked and its implemention can be downloaded [here](https://pypi.org/project/orderedset/#files) and be installed by:
```terminal
pip3 install orderedset
``` 
> Python lists are arrays exponentially over-allocated, surprisingly.

This Orderedset offers O(1) for add, remove, and contains. It is coded in C, and its doubly linked nodes [prev, next, value] are lists. Other data structures would benefit specific scenarios.

### Creating new corpus, suitable for NLTK library

- Code for filename formating
- The corpus implementation

In [5]:
from string import punctuation as punct
from nltk.corpus import stopwords as stop
from nltk.tokenize import word_tokenize

def create_unix_filename(s, limit, ext):
    name = [lemma for lemma in word_tokenize(s[:limit].lower()) 
            if lemma not in punct and stop.words('english')
           if len(lemma)>3] #removes "'s", "in", "the", etc
    return ('-'.join(name)+ext)

def create_corpus(folder, dataframe):
    for i, row in dataframe.iterrows():
        ident=row['id']; title=row['title']
        content=row['content']
        
        limit=60 #character limit for fileids (less than or equal 255 characters for unix)
        article=str(ident)+'_'+create_unix_filename(title,limit,'.txt') #translate title for results below

        corpus=open(folder+'/'+article,'a')
        corpus.write(' [ '+title+' ] '+content)
        corpus.close()

- Make folder for storing the corpus

In [3]:
%%sh
#!/bin/sh

dir=./dataset
mkdir -p $dir/portuguese_council
ls $dir

news__ptbr.csv
portuguese_council


In [6]:
%%timeit

create_corpus('./dataset/portuguese_council/', politicnews)


1 loop, best of 5: 8.43 s per loop


- Instantiate corpus reader for NLTK

In [7]:
from nltk.corpus.reader import PlaintextCorpusReader

corpus=PlaintextCorpusReader('./dataset/portuguese_council/', r'.*\.txt')

len(corpus.fileids())

7643

First 15 fileids (troublesome " 's " already removed and translation altogether):
<pre>├── 1002_aécio-is-the-best-voted-tucano-in-sp.txt
├── 1004_2018-looking-like-2006.txt
├── 1032_social-division-may-benefit-brazil-defends-political-scientist.txt
├── 1046_aécio-will-accompany-verification-at-her-sister-&apos;s-house-in-bh.txt
├── 1064_aécio-says-he-expects-youssef-to-tell-everything-he-knows.txt
├── 1079_aécio-if-elected-first-mission-is-to-unify-the-country.txt
├── 1094_aécio-votes-in-bh-and-talks-about-unifying-the-country.txt
├── 1114_cid-&apos;s-godson-turns-the-game-over-the-favorite.txt
├── 1128_lobby-agenda-is-waiting-for-the-president.txt
├── 1131_the-election-in-which-it-is-urgent-to-have.txt
├── 1133_aecio-reacts-and-also-uses-his-toolbox.txt
├── 1151_aécio-neves-says-he-will-fire-petrobras-board-if-elected.txt
├── 1160_ácio-accuses-pt-of-terrorism-dilma-highlights-pronatec.txt
├── 1186_aécio-and-dilma-arrive-at-the-last-debate-without-trying-cup.txt
├── 1208_agreement-is-initial-act-of-political-reform-says-janot.txt
(...)</pre>

Use this command for cleansing:

```sh
find ./portuguese_council/ -maxdepth 1 -name "*.txt" -print0 | xargs -0 rm
```


#### The inverted index for indexing news (or articles) 

In [8]:
import nltk, collections 
from orderedset import OrderedSet

def inverted_index(corpus):
    i = collections.defaultdict(OrderedSet)
    for filename in corpus.fileids():
        for term in corpus.words(filename):
            i[term.lower()].add(filename)
    return i

#### Search for testing the inverted index (only)

In [43]:
# Warning: This do not replace boolean syntactic analyzers.

def search(i, query):
    """
    :param i: inverted index, case insenstive
    :param query: user input, patterns:
       1-term: if found, a term is returned;
       term1 AND term2: all query terms;
       term1 OR term2: any of the query terms.
    """
    t=lambda s: s.lower().strip()   
    if ' OR ' in query:
        term = query.split(' OR ')
        return i[t(term[0])].union(i[t(term[1])]) 
    elif ' AND ' in query:
        term = query.split(' AND ')
        return i[t(term[0])].intersection(i[t(term[1])])
    else:
        return i[t(query)]  #case insensitive  

In [10]:
%%timeit

# Generating the indexes for all news
i = inverted_index(corpus)

1 loop, best of 5: 16.5 s per loop


In [12]:
# Running again for having the index to perform the tests below
i = inverted_index(corpus)

### Testing (case sensitive)

In [33]:
from orderedset import OrderedSet

c=i["Campina"]
g=i['Grande']

len(c.intersection(g)) # Must be 12 occurrences


12

### Testing (case insensitive)

In [28]:

assert (len(search(i,"Belo AND Horizonte")) ==  len(search(i,"belo AND horizonte")))

In [45]:
assert (len(search(i," latin ")) == len(search(i,"Latin")))

#### Occurences of word "Basque" 

In [46]:
search(i, 'Basque OR Vascos')

OrderedSet()

In [66]:
search(i, 'Basques')

OrderedSet()

#### Occurrences of "Latin"

In [47]:
len(search(i, '                 Latin'))

5

In [48]:
len(search(i, 'Latino'))

20

#### Occurrences

In [50]:
len(search(i, 'Italiano'))

50

In [51]:
len(search(i, 'falso OR falsa')) 

92

In [52]:
len(search(i, 'Italiano OR Italiana')) 

73

#### Using orderedset in NLTK corpus

In [53]:
%%timeit

# Bonus Exercise: Using orderedset, compute hit list for ((paris AND NOT france) OR lear)
def dsearch(inverted_index):
    files_with_paris = inverted_index['Paris']
    files_with_france = inverted_index['France']
    files_with_lear = inverted_index['Lear']
    return ((files_with_paris.difference(files_with_france)).union(files_with_lear))

i = inverted_index(nltk.corpus.gutenberg)

print(dsearch(i))

OrderedSet()
OrderedSet()
OrderedSet()
OrderedSet()
OrderedSet()
OrderedSet()
1 loop, best of 5: 2.92 s per loop


#### Conslusions

Boolean search is a simplistic approach yet powerful. The case-sensitive approach showed faster indexation, ~8s against ~16s of its oppose. It was also possible to search for "Englishman", which returned no results, while its opposed returned 30 occurences of unrelated subjects. 


>We ran this search in an important communication media of the brazilian society.


It follows the search for "Polish", although its existance in the culturalization processs for finding any result of this word. #all case-insensitive

In [56]:
len(search(i, 'polonês OR polish')) 

3

In [57]:
len(search(i, 'insanidade OR sanidade')) 

2

In [60]:
len(search(i, 'Holandês')) #case insenstive ,

6

In [62]:
search(i, 'England')

OrderedSet()

In [63]:
search(i, 'Reino Unido')

OrderedSet()

In [64]:
search(i,'Grã-Bretanha')

OrderedSet()

In [65]:
len(search(i,'Espanha'))

22

### Missing features and improvements

- [ ] Reduce the number of syscalls by reducing files and unifying achieves
- [x] Reduce filenames by removing words with size less than or equal 3 characters

### References and Thanks
- Leandro Balby for have required and provided the dataset.
- Joe James for have provided an overview of NLTK library.
